# PASCAL VOC Dataset

* [TensforFlwo Datasets - voc](https://www.tensorflow.org/datasets/catalog/voc)

> PASCAL Visual Object Classes Challenge, corresponding to the Classification and Detection competitions.

<img src="image/pascal_voc_xml_example.png" align="left" width=350/>

In [1]:
from typing import (
    Tuple,
)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

In [9]:
import sys
sys.path.append("../../lib")

In [11]:
%load_ext autoreload
%autoreload 2

from util_tf.tfds import (
    convert_pascal_voc_bndbox_to_yolo_bbox
)

# TensorFlow Datasets Pascal VOC

NOTE: TFDS BBox format is (y_min, x_min, y_max, x_max) and normalized with image size.

* [tfds.features.BBox](https://www.tensorflow.org/datasets/api_docs/python/tfds/features/BBox)

```
tfds.features.BBox(
    ymin, xmin, ymax, xmax
)
```

* [datasets/tensorflow_datasets/object_detection/voc.py](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/object_detection/voc.py#L89-121)

```
def _get_example_objects(annon_filepath):
  """Function to get all the objects from the annotation XML file."""
  with tf.io.gfile.GFile(annon_filepath, "r") as f:
    root = xml.etree.ElementTree.parse(f).getroot()

    # Disable pytype to avoid attribute-error due to find returning
    # Optional[Element]
    # pytype: disable=attribute-error
    size = root.find("size")
    width = float(size.find("width").text)
    height = float(size.find("height").text)

    for obj in root.findall("object"):
      # Get object's label name.
      label = obj.find("name").text.lower()
      # Get objects' pose name.
      pose = obj.find("pose").text.lower()
      is_truncated = obj.find("truncated").text == "1"
      is_difficult = obj.find("difficult").text == "1"
      bndbox = obj.find("bndbox")
      xmax = float(bndbox.find("xmax").text)
      xmin = float(bndbox.find("xmin").text)
      ymax = float(bndbox.find("ymax").text)
      ymin = float(bndbox.find("ymin").text)
      yield {
          "label": label,
          "pose": pose,
          "bbox": tfds.features.BBox(
              ymin / height, xmin / width, ymax / height, xmax / width
          ),
          "is_truncated": is_truncated,
          "is_difficult": is_difficult,
      }
```

In [ ]:
ds, info = tfds.load(
    name='voc', 
    split='train',
    data_dir="/content/gdrive/MyDrive/git/data",
    with_info=True,
)
fig = tfds.show_examples(ds, info)

In [13]:
bbox = tfds.features.BBox(
    ymin=0.0, xmin=0.0, ymax=1.1, xmax=1.1
)

In [15]:
type(bbox.ymin)

float

In [57]:
dataset = tf.data.Dataset.from_tensor_slices([
    tf.constant([0.14375, 0.0437018, 0.97083336], dtype=np.float32),
    tf.constant([0.14583333, 0.24164525, 0.57916665], dtype=np.float32),
    tf.constant([0.6, 0.5244216, 0.8541667], dtype=np.float32),
])

In [63]:
def f(x):
    y = tf.concat([x[0], x[1]], axis=-1)
    return y

for d in dataset.map(f):
    print(d)

ValueError: in user code:

    File "/var/folders/_4/8v285hqs45xfzk0l1nlr3yq40000gn/T/ipykernel_95641/4049044319.py", line 2, in f  *
        y = tf.concat([x[0], x[1]], axis=-1)

    ValueError: Can't concatenate scalars (use tf.stack instead) for '{{node concat}} = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32](strided_slice, strided_slice_1, concat/axis)' with input shapes: [], [], [].
